In [5]:
%%writefile prepare.py

#Input for this software are a configuration file and a survey schedule in csv format.
import os
import glob
import pandas as pd
pd.set_option('display.width',512)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime
import mmap
import sys
import gc

def find(indexfile,start,end):
    print indexfile,start,end  
    dtypes={'start_seconds':np.float64,'start_ttime':'str','freq':np.float64,'filename':'str','start_byte':'int64','end_byte':'int64'}
    aa = pd.read_csv(indexfile,dtype=dtypes)

    aa.dropna(inplace=True)
    aa.set_index('start_seconds',inplace=True,drop=True)
    
    print aa.head(20)
    df = aa[(aa.index >= start) & (aa.index <= end)]
    print df.head(20)
    return df

def sec2time(sec, n_msec=3):
    ''' Convert seconds to 'D days, HH:MM:SS.FFF' '''
    if hasattr(sec,'__len__'):
        return [sec2time(s) for s in sec]
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)
    if n_msec > 0:
        pattern = '%%02d:%%02d:%%0%d.%df' % (n_msec+3, n_msec)
    else:
        pattern = r'%02d:%02d:%02d'
    if d == 0:
        return pattern % (h, m, s)
    return ('%d days, ' + pattern) % (d, h, m, s)

def time2sec(time, n_msec=3):
    ''' Convert 'D days, HH:MM:SS.FFF' to seconds'''
    from datetime import datetime as dt
    pt=dt.strptime(time,'%H:%M:%S.%f')
    total_seconds=pt.second+pt.minute*60+pt.hour*3600
    return total_seconds

def findposition(Tstart,Tend,filename,first=None,last=None):
    print 'Tstart',Tstart
    if first and last:
        sys.exti(0)
        
    fsize=os.path.getsize(filename) #size of file (in bytes)  
    print 'File:',filename
    
    if fsize == 0:
        print ",,",filename
        sys.exit(1)
    
    print 'Fsize',fsize
    start=0
    end=1468
    ti=readblocks2(filename,start,end)
    print 'Tinicial',ti
    
    start=fsize-1468
    end=fsize
    tf=readblocks2(filename,start,end)
    print 'Tfinal',tf
    
    deltaT=(tf[0]-ti[0])/(fsize/1468)
    print 'DeltaT',deltaT

    if first:
        ## start - Tstart
        diffT=(tf[0] - Tstart)
        print 'DiffT',diffT
        nblocks=int(diffT/deltaT)
        start=fsize-(nblocks+2)*1468
        end=start+1468
        
        print 'nblocks',nblocks

        print 'Fsize',fsize,'start',start,'end',end
        tt=readblocks2(filename,start,end)
        tt.extend([start,fsize])
    
    if last:
        ## Tend-end
        diffT=(Tend-ti[0]) 
#         print diffT
        nblocks=int(diffT/deltaT)
        start=nblocks*1468
        end=(nblocks+2)*1468
        tt=readblocks2(filename,start,end)
        tt.extend([0,end])
    
    #print tt
    return tt

def filterdata(indexfile,start,end):
    ### Search files corresponding to the valid survey time
    start=time2sec(start)
    end=time2sec(end)
    
    df=find(indexfile,start,end)
    if len(df.index.values) == 0:
        start=start-15
        end=end+15
        df=find(indexfile,start,end)
    
    ### Separate the Polarization file list
    
    E1=df[df['filename'].str.contains('_E1_', case=True, flags=0, na=np.nan, regex=True)]
    E2=df[df['filename'].str.contains('_E2_', case=True, flags=0, na=np.nan, regex=True)]
        
    new=[]
    ### If there is some seconds in the previous file
    print E1
    if E1.index.values[0] > start:
        print 'Some seconds in the previous file' 
        filename=E1.iloc[0]['filename']
        numberfile=int(filename.split('_')[-1])-1
        if not numberfile == 0:
            prevfile='_'.join(filename.split('_')[:-1]+['{0:04}'.format(numberfile)])
            new=[findposition(start,end,prevfile,first=True,last=False)]
            E1.reset_index(level=0,inplace=True)
            result=E1.values.tolist()
            del E1
            gc.collect()
            dtypes={'start_seconds':np.float64,'start_ttime':'str','freq':np.float64,'filename':'str','start_byte':'int64','end_byte':'int64'}
            labels=['start_seconds','start_ttime','freq','filename','start_byte','end_byte']
            new.extend(result)
            E1=pd.DataFrame.from_records(new,columns=labels)
            E1.set_index('start_seconds',inplace=True,drop=True)
            
    new=[]
    nextfile=(E1['filename'].tail(1)).values
    IX=(E1['filename'].tail(1)).index
#     print IX
#     print nextfile[0]
    new=findposition(start,end,nextfile[0],first=False,last=True)
#     print new[-1]
#     E1.set_value(IX, 'end_byte', new[-1])
    E1.loc[(E1['filename'].tail(1)).index,  'end_byte'] = new[-1]
    print E1
     
    new=[]
    ### If there is some seconds in the previous file  
    print E2
    if E2.index.values[0] > start:
        print 'Some seconds in the previous file' 
        filename=E2.iloc[0]['filename']
        numberfile=int(filename.split('_')[-1])-1
        if not numberfile == 0:
            prevfile='_'.join(filename.split('_')[:-1]+['{0:04}'.format(numberfile)])
            new=[findposition(start,end,prevfile,first=True,last=False)]
            E2.reset_index(level=0,inplace=True)
            result=E2.values.tolist()
            del E2
            gc.collect()
            dtypes={'start_seconds':np.float64,'start_ttime':'str','freq':np.float64,'filename':'str','start_byte':'int64','end_byte':'int64'}
            labels=['start_seconds','start_ttime','freq','filename','start_byte','end_byte']
            new.extend(result)
            E2=pd.DataFrame.from_records(new,columns=labels)
            E2.set_index('start_seconds',inplace=True,drop=True)
            
    new=[]
    nextfile=(E2['filename'].tail(1)).values
    IX=(E2['filename'].tail(1)).index
#     print IX
#     print nextfile[0]
    new=findposition(start,end,nextfile[0],first=False,last=True)
#     print new[-1]
#     E1.set_value(IX, 'end_byte', new[-1])
    E2.loc[(E2['filename'].tail(1)).index,  'end_byte'] = new[-1]
    print E2
    
    return [E1,E2]

def readheader(BS,hprint=None):
    from bitstring import BitStream
    import struct
    b = BitStream('0x'+''.join(x.encode('hex') for x in struct.unpack('>72s',BS)))
    #magic = b.read(32).hex# 32 magic word
    header = {}
    header["recordlength"] = b.read(16).uint   # 16 0..65535
    header["hdrlen"] = b.read(8).uint    # 8 0..255
    header["blocksize"] = b.read(8).uint   # 8 0..25
    header["samplerate"] = b.read(16).uint   # 16 0..65535
    header["efegain"] = b.read(10).uint   # 10 0..650
    header["qu"] = b.read(3).uint    # 16 0..7 (0=>1bits,1=>2bits,2=>4bits,4=>8bits,5=>16bits, 3,6,7 spare)
    header["msg"] = b.read(3).uint    # 3 6
    header["frameid"] = b.read(32).uint  # 32 0..4294967295
    header["version"] = b.read(7).uint    # 7 0..127
    header["timetag_samps"] = b.read(25).uint   # 25 0..17499999
    header["offsetfreq"] = b.read(32).int   # 32 0..4294967295
    header["timetag_secs"] = b.read(17).uint   # 17 0..86399
    header["subc"] = b.read(4).uint    # 4 0..16
    header["digitalgain"] = b.read(11).uint   # 11 0..2047
    header["subchan0_offset"] = b.read(32).int    # 32 -2147483647..+2147483647
    header["subchan1_offset"] = b.read(32).int    # 32 -2147483647..+2147483647
    header["subchan2_offset"] = b.read(32).int    # 32 -2147483647..+2147483647
    header["subchan3_offset"] = b.read(32).int    # 32 -2147483647..+2147483647
    header["sweeprate"] = b.read(32).int    # 32 -2147483647..+2147483647
    header["path_delay"] = b.read(32).int    # 32 0..2^32-1
    header["gdspid"] = b.read(8).uint    # 1 0..1
    header["hs"] = b.read(1).uint    # 1 0..1
    header["semr"] = b.read(12).int    # 12
    header["sweepchange"] = b.read(11).uint   # 11 0..2047
    header["ncov"] = b.read(1).uint    # 1 0..1
    header["ncoreset_c"] = b.read(11).int    # 11 -1024..+1024
    header["ncoreset_t"] = b.read(20).uint   # 20 0..863999
    b.read(128).uint    # 128 Empty
    
    if hprint:
        #print header
        print yaml.dump(header, default_flow_style=False)
    
    return header

def readblocks2(filename,start,end):
    with open(filename, 'rb') as f:
        fsize=os.path.getsize(filename)
        if not fsize == 0:
            # memory-map the file, size 0 means whole file
            mm = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)
        else:
            return ['NaN','NaN','NaN',filename]

        mm.seek(start)
        position=mm.tell()
#         print position
            
        while True:
            # Magic Word
            byte=mm.read(4)
            if not byte:
                break
            # Read Header (check cuantization and save timestamp)
            header = readheader(mm.read(72),hprint=None)
            ## Calculate UTC
            X=np.float64(header["timetag_samps"])*np.float64(1./17500000)
            Y=np.float64(header["path_delay"])*np.float64(1./35000000)
            utctime=np.float64(header["timetag_secs"])+X-Y
            ttime=sec2time(utctime,6)
            fs=header["samplerate"]
#             print utctime,',',ttime,',',filename
            mm.read(1392)
            if end == mm.tell():
                return [utctime,ttime,fs,filename]

def readblocks(filename):
    import mmap

    with open(filename, 'rb') as f:
        fsize=os.path.getsize(filename)
        if not fsize == 0:
            # memory-map the file, size 0 means whole file
            mm = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)
        else:
            return ['NaN','NaN','NaN',filename,'0',fsize]

        while True:
            # Magic Word
            byte=mm.read(4)
            if not byte:
                break
            # Read Header (check cuantization and save timestamp)
            header = readheader(mm.read(72),hprint=None)
            ## Calculate UTC
            X=np.float64(header["timetag_samps"])*np.float64(1./17500000)
            Y=np.float64(header["path_delay"])*np.float64(1./35000000)
            utctime=np.float64(header["timetag_secs"])+X-Y
            ttime=sec2time(utctime,6)
            fs=header["samplerate"]
            mm.read(1392)
            return [utctime,ttime,fs,filename,int(0),int(fsize)]
            break

def genindex(indexfile,workdir):
    import csv
    thelist=sorted(glob.glob(workdir+'/*'))
    ll=[]
    for i in thelist:
        if i.split('_')[-1] == '0000':
            continue
        line=readblocks(i)
        ll.append(line)
    with open(indexfile, 'wa') as f:
        fieldnames = ['start_seconds','start_ttime','freq','filename','start_byte','end_byte']
        writer = csv.writer(f)
        writer.writerow(fieldnames)
        writer.writerows(ll)

## Variable 

### Schedule from the Observing Report
schedulefile='/home/taller-dis1/Angel/new_process/schedule.csv'
configfile='/home/taller-dis1/Angel/new_process/prepare.conf'
#dataschedulefile='/home/taller-dis1/Angel/new_process/dataschedule.csv'
workdir='/home/taller-dis1/Angel/new_process/testing/'

surveypath='/home/taller-dis1/Angel/ESA/'

indexfiles=['ESU1_index.csv','ESU2_index.csv','ESU3_index.csv']

## Generate First Index for the Survey
for i in indexfiles:
    indexfile=workdir+i
    print indexfile
    if os.path.isfile(indexfile):
        print 'Index OK'
    else: 
        print 'Index file doesn exist.'
        print 'Creating...',indexfile
        genindex(indexfile,surveypath+i.split('_')[0])

## Read Schedule
headers = ['datetime', 'recordlength', 'source','frec_hz','resolution']
dtypes = {'datetime': 'str', 'recordlength': 'float', 'source': 'str','frec_hz':'float', 'resolution':'int'}
df = pd.read_csv(schedulefile, header=None, names=headers, dtype=dtypes)
#print df

## Generate Data Schedule
for idx in range(0,3,1):
    for index, row in df.iterrows():
        #print row
        tstart=datetime.datetime.strptime(row['datetime'],'%Y/%m/%d/%H:%M:%S')
        tend=tstart+datetime.timedelta(seconds=row['recordlength'])
        #     print tstart.timetuple().tm_yday, tstart.strftime("%H:%M:%S.%f"),tend.strftime("%H:%M:%S.%f")
        
        GDSP1,GDSP2=filterdata(workdir+indexfiles[idx],tstart.strftime("%H:%M:%S.%f"),tend.strftime("%H:%M:%S.%f"))
        
        GDSP1.to_csv(workdir+indexfiles[idx]+'gdsp1.csv', mode='a', header=False)
        GDSP2.to_csv(workdir+indexfiles[idx]+'gdsp2.csv', mode='a', header=False)        


Overwriting prepare.py


In [ ]:
%run prepare